In [578]:
from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
import pandas

In [579]:
data_orange_pi_st = pandas.read_csv("sbc_performance_data/orange_pi/zero_lts/buster/singlethread_output.csv")
data_raspberry_pi_st = pandas.read_csv("sbc_performance_data/raspberry_pi/buster/4B/std_kernel_4B_singlethread_output.csv")
data_rockpi_st = pandas.read_csv("sbc_performance_data/rockpi_s/stretch_4.4.143/singlethread_output.csv")

#remove last 5 rows of rockpi dataset to make consistent with remaining datasets 
data_rockpi_st = data_rockpi_st.loc[0:44]

device_dfs = [data_orange_pi_st,data_raspberry_pi_st,data_rockpi_st]

[dfs.reset_index(inplace=True) for dfs in device_dfs]

for dev in device_dfs:
    dev['time'] -= dev['time'][0]


data_orange_pi_st['color'] = 'Orange'
data_raspberry_pi_st['color'] = 'Crimson'
data_rockpi_st['color'] = 'Grey'

data_orange_pi_st['CPU'] = '4x Cortex-A7'
data_raspberry_pi_st['CPU'] = '4x Cortex-A72'
data_rockpi_st['CPU'] = '2x Cortex-A72 + 4x Cortex-A53'

data_orange_pi_st['device'] = 'orange_pi'
data_raspberry_pi_st['device'] = 'raspberry_pi'
data_rockpi_st['device'] = 'rockpi'






In [580]:
for dev in device_dfs:
    dev['time'] -= dev['time'][0]
    
#Orange PI
#Time for device to solve queens
data_orange_pi_st_timestamp = data_orange_pi_st.time/60

#Duration of experiment
data_orange_pi_st_time = data_orange_pi_st.seconds + data_orange_pi_st.microseconds/1000000

#Avg temp cpu+gpu
data_orange_pi_st_avg_temp = (data_orange_pi_st.cpu_temp + data_orange_pi_st.gpu_temp)/2

#add columns to dataframe
data_orange_pi_st['duration'] = data_orange_pi_st_time
data_orange_pi_st['time_to_solve'] = data_orange_pi_st_timestamp
data_orange_pi_st['avg_temp'] = data_orange_pi_st_avg_temp

#Raspberry PI
#Time for device to solve queens
data_raspberry_pi_st_timestamp = data_raspberry_pi_st.time/60

#Duration of experiment
data_raspberry_pi_st_time = data_raspberry_pi_st.seconds + data_raspberry_pi_st.microseconds/1000000

#Avg temp cpu+gpu
data_raspberry_pi_st_avg_temp = (data_raspberry_pi_st.cpu_temp + data_raspberry_pi_st.gpu_temp)/2

data_raspberry_pi_st['duration'] = data_raspberry_pi_st_time
data_raspberry_pi_st['time_to_solve'] = data_raspberry_pi_st_timestamp
data_raspberry_pi_st['avg_temp'] = data_raspberry_pi_st_avg_temp

#Rockpi
#Time for device to solve queens
data_rockpi_st_timestamp = data_rockpi_st.time/60

#Duration of experiment
data_rockpi_st_time = data_rockpi_st.seconds + data_rockpi_st.microseconds/1000000

#Avg temp cpu+gpu
data_rockpi_st_avg_temp = (data_rockpi_st.cpu_temp + data_rockpi_st.gpu_temp)/2

data_rockpi_st['duration'] = data_rockpi_st_time
data_rockpi_st['time_to_solve'] = data_rockpi_st_timestamp
data_rockpi_st['avg_temp'] = data_rockpi_st_avg_temp


orange_ds = ColumnDataSource(data_orange_pi_st) 
raspberry_pi_ds = ColumnDataSource(data_raspberry_pi_st)
rockpi_ds = ColumnDataSource(data_rockpi_st)


In [581]:

#fig.extra_y_ranges = {"foo1": Range1d(start=0, end=50), "foo2": Range1d(start=0, end=25)}


In [582]:
x = data_raspberry_pi_st['index']
y = data_raspberry_pi_st['microseconds']

o_x = data_orange_pi_st['index']
o_y = data_orange_pi_st['microseconds']

r_x = data_rockpi_st['index']
r_y = data_rockpi_st['microseconds']

In [583]:
xmin = x.min()
oxmin = o_x.min()
rxmin = r_x.min()

x_o_x_time = x.apply(lambda x: x - xmin).combine_first(o_x.apply(lambda x: x - oxmin)).combine_first(r_x.apply(lambda x: x - rxmin))

In [584]:
output_notebook()

source = ColumnDataSource(data=dict(
    x=x_o_x_time,
    y1=y,
    y2=o_y,
    y3=r_y,
))

p = figure()
p.add_tools(HoverTool())
p.background_fill_color = "grey"
p.vbar_stack(['y1','y2','y3'], x='x', width=0.8, color=("red", "orange", "yellow"), source=source)
#p.scatter(x = 'x1', y='y',size=20, source=source)
show(p)


Loading BokehJS ...

In [589]:
f = figure()

#title
f.title.text="12 Queens - Singlethreaded SBC perf"
f.title.align="center"

#instantiate and confiture hover tool 
htool = HoverTool()
htool.tooltips = [
    ("cpu_temp", "@cpu_temp"),
    ("gpu_temp", "@gpu_temp"),
    ("CPU","@CPU"),
    ("Time to solve","@duration")
]
f.add_tools(htool)

f.line(x='index', y="cpu_temp",line_width = 2, line_color="Orange", source=orange_ds,legend_label='orange_ds: cpu temp')
f.line(x='index', y="cpu_temp",line_width = 2, line_color="Grey", source=rockpi_ds,legend_label='rockpi_ds: cpu temp')
f.line(x='index', y="cpu_temp",line_width = 2, line_color="Crimson", source=raspberry_pi_ds,legend_label='raspberry pi: cpu temp')

f.line(x='index', y="gpu_temp",line_width = 2, line_dash = 'dashed', line_color="Orange", source=orange_ds,legend_label='orange_ds gpu temp')
f.line(x='index', y="gpu_temp",line_width = 2, line_dash = 'dashed', line_color="Grey", source=rockpi_ds,legend_label='rockpi_ds gpu temp')
f.line(x='index', y="gpu_temp",line_width = 2, line_dash = 'dashed', line_color="Crimson", source=raspberry_pi_ds,legend_label='raspberry pi gpu temp')

#time to solve
f.line(x='index', y="duration",line_width = 2, line_dash = 'dotted', line_color="Orange", source=orange_ds,legend_label='raspberry pi time to solve s')
f.line(x='index', y="duration",line_width = 2, line_dash = 'dotted', line_color="Grey", source=rockpi_ds,legend_label='raspberry pi time to solve s')
f.line(x='index', y="duration",line_width = 2, line_dash = 'dotted', line_color="Crimson", source=raspberry_pi_ds,legend_label='raspberry pi time to solve s')


#TODO: implement filtering of indices
#TODO: labels
#TODO: style legend

show(f)

In [586]:
data_orange_pi_mt = pandas.read_csv("sbc_performance_data/orange_pi/zero_lts/buster/multithread_output.csv")
data_raspberry_pi_mt = pandas.read_csv("sbc_performance_data/raspberry_pi/buster/4B/std_kernel_4B_multithread_output.csv")
data_rockpi_mt = pandas.read_csv("sbc_performance_data/rockpi_s/stretch_4.4.143/multithread_output.csv")

device_dfs_mt = [data_orange_pi_mt,data_raspberry_pi_mt,data_rockpi_mt]

[dfs.reset_index(inplace=True) for dfs in device_dfs]

for dev in device_dfs_mt:
    dev['time'] -= dev['time'][0]

data_orange_pi_mt['color'] = 'Orange'
data_raspberry_pi_mt['color'] = 'Crimson'
data_rockpi_mt['color'] = 'Grey'

data_orange_pi_mt['CPU'] = '4x Cortex-A7'
data_raspberry_pi_mt['CPU'] = '4x Cortex-A72'
data_rockpi_mt['CPU'] = '2x Cortex-A72 + 4x Cortex-A53'

data_orange_pi_mt['device'] = 'orange_pi'
data_raspberry_pi_mt['device'] = 'raspberry_pi'
data_rockpi_mt['device'] = 'rockpi'

#titles
f.title.text="12 Queens - Multihreaded SBC perf"
f.title.align="center"

#copy cpu temp and adjust scale for better visualization 
#temp_visualization_scale = 0.5
#data_orange_pi_mt['cpu_temp_viz'] = data_orange_pi_mt['cpu_temp'].apply(lambda x: x * temp_visualization_scale)

#Orange PI
#Time for device to solve queens
data_orange_pi_mt_timestamp = data_orange_pi_mt.time/60

#Duration of experiment
data_orange_pi_mt_time = data_orange_pi_mt.seconds + data_orange_pi_mt.microseconds/1000000

#Avg temp cpu+gpu
data_orange_pi_mt_avg_temp = (data_orange_pi_mt.cpu_temp + data_orange_pi_mt.gpu_temp)/2

#add columns to dataframe
data_orange_pi_mt['duration'] = data_orange_pi_mt_time
data_orange_pi_mt['time_to_solve'] = data_orange_pi_mt_timestamp
data_orange_pi_mt['avg_temp'] = data_orange_pi_mt_avg_temp

#Raspberry PI
#Time for device to solve queens
data_raspberry_pi_mt_timestamp = data_raspberry_pi_mt.time/60

#Duration of experiment
data_raspberry_pi_mt_time = data_raspberry_pi_mt.seconds + data_raspberry_pi_mt.microseconds/1000000

#Avg temp cpu+gpu
data_raspberry_pi_mt_avg_temp = (data_raspberry_pi_mt.cpu_temp + data_raspberry_pi_mt.gpu_temp)/2

data_raspberry_pi_mt['duration'] = data_raspberry_pi_mt_time
data_raspberry_pi_mt['time_to_solve'] = data_raspberry_pi_mt_timestamp
data_raspberry_pi_mt['avg_temp'] = data_raspberry_pi_mt_avg_temp

#Rockpi
#Time for device to solve queens
data_rockpi_mt_timestamp = data_rockpi_mt.time/60

#Duration of experiment
data_rockpi_mt_time = data_rockpi_mt.seconds + data_rockpi_mt.microseconds/1000000

#Avg temp cpu+gpu
data_rockpi_mt_avg_temp = (data_rockpi_mt.cpu_temp + data_rockpi_mt.gpu_temp)/2

data_rockpi_mt['duration'] = data_rockpi_mt_time
data_rockpi_mt['time_to_solve'] = data_rockpi_mt_timestamp
data_rockpi_mt['avg_temp'] = data_rockpi_mt_avg_temp

#Bokeh specific data sources
orange_mt_ds = ColumnDataSource(data_orange_pi_mt) 
raspberry_pi_mt_ds = ColumnDataSource(data_raspberry_pi_mt)
rockpi_mt_ds = ColumnDataSource(data_rockpi_mt)
data_orange_pi_mt.head()

,time,queen,threads,solutions,seconds,microseconds,cpu_temp,gpu_temp,color,CPU,device,duration,time_to_solve,avg_temp
0,0,12,4,14200,66,998141,30.37,30.37,Orange,4x Cortex-A7,orange_pi,66.998141,0.000000,30.37
1,68,12,4,14200,67,226209,36.66,36.66,Orange,4x Cortex-A7,orange_pi,67.226209,1.133333,36.66
2,134,12,4,14200,66,713568,43.68,43.68,Orange,4x Cortex-A7,orange_pi,66.713568,2.233333,43.68
3,201,12,4,14200,66,652859,47.67,47.67,Orange,4x Cortex-A7,orange_pi,66.652859,3.350000,47.67
4,268,12,4,14200,66,824575,48.64,48.64,Orange,4x Cortex-A7,orange_pi,66.824575,4.466667,48.64


In [587]:
g = figure()

g.add_tools(htool)

#def style_line(figline,x,y,width,source,dash,color,label):
#    return (figline(x=x, y=y,line_width = width, line_dash = dash, line_color=color, source=source,legend_label=label))
#g.line = style_line(g.line,'index','cpu_temp',2,"solid","Orange",orange_mt_ds,'orange_ds: cpu temp')

#cpu temp graph
g.line(x='index', y="cpu_temp",line_width = 2, line_color="Orange", source=orange_mt_ds,legend_label='orange_ds: cpu temp')
g.line(x='index', y="cpu_temp",line_width = 2, line_color="Grey", source=rockpi_mt_ds,legend_label='rockpi_ds: cpu temp')
g.line(x='index', y="cpu_temp",line_width = 2, line_color="Crimson", source=raspberry_pi_mt_ds,legend_label='raspberry pi: cpu temp')


#gpu temp graph
g.line(x='index', y="gpu_temp",line_width = 2, line_dash = 'dashed', line_color="Orange", source=orange_mt_ds,legend_label='orange_ds gpu temp')
g.line(x='index', y="gpu_temp",line_width = 2, line_dash = 'dashed', line_color="Grey", source=rockpi_mt_ds,legend_label='rockpi_ds gpu temp')
g.line(x='index', y="gpu_temp",line_width = 2, line_dash = 'dashed', line_color="Crimson", source=raspberry_pi_mt_ds,legend_label='raspberry pi gpu temp')

#time to solve
g.line(x='index', y="duration",line_width = 2, line_dash = 'dotted', line_color="Orange", source=orange_mt_ds,legend_label='raspberry pi time to solve s')
g.line(x='index', y="duration",line_width = 2, line_dash = 'dotted', line_color="Grey", source=rockpi_mt_ds,legend_label='raspberry pi time to solve s')
g.line(x='index', y="duration",line_width = 2, line_dash = 'dotted', line_color="Crimson", source=raspberry_pi_mt_ds,legend_label='raspberry pi time to solve s')


#TODO: implement filtering of indices

show(g)